In [36]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

### Todo lo que se va a leer en este notebook fue inspirado por https://elitedatascience.com/imbalanced-classes

In [37]:
df1 = pd.read_csv('data/encoding_stats_features.csv', index_col='person')
df2 = pd.read_csv('data/viewed_product_stats.csv', index_col='person')
df3 = pd.read_csv('data/features_basicas.csv', index_col='person')
df4 = pd.read_csv('data/days_elapsed_from_last_event.csv', index_col='person')
df5 = pd.read_csv('data/eventos_en_los_ultimos_15_dias.csv', index_col='person')

labels = pd.read_csv('data/labels_training_set.csv', index_col='person')
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 6 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
dtypes: float64(5), int64(1)
memory usage: 2.1+ MB


In [38]:
# joineando df1 con df3
df_final = df3.join(df1, how='left', on='person')
df_final = df_final.fillna(df_final.mean())

In [39]:
df_final = df_final.join(df2, how='left', on='person')
df_final = df_final.fillna(0)

In [40]:
df_final = df_final.join(df4, how='left', on='person')
df_final = df_final.fillna(df_final.max())
df_final = df_final.join(df5, how='left', on='person')
df_final = df_final.fillna(0)

In [41]:
df_features = df_final.copy()
df_final = df_final.join(labels, how='inner', on='person')

In [42]:
df_final['label'].value_counts(normalize=True)

0    0.949521
1    0.050479
Name: label, dtype: float64

## Clases desbalanceadas

Podemos observar claramente que las clases estan desbalanceadas, vamos a probar distintos métodos para balancearlas

In [43]:
from sklearn.utils import resample

df_no_compradores = df_final[df_final.label==0]
df_compradores = df_final[df_final.label==1]

print(df_no_compradores.label.value_counts())
print(df_compradores.label.value_counts())

0    18434
Name: label, dtype: int64
1    980
Name: label, dtype: int64


#### Up-Sample de compradores: 

Primero voy a probar hacer un up-sample de los compradores, para no achicar mucho el set de datos.

In [45]:
proporcion_compradores = 0.5

df_mas_compradores = resample(df_compradores, 
                              replace=True,
                              n_samples=int(df_no_compradores.label.value_counts()[0] * proporcion_compradores),
                              random_state=123)
df_upsample = pd.concat([df_mas_compradores, df_no_compradores])

#### Down-Sample de no compradores:

Ahora voy a quitar una cierta cantidad de compradores para quedarme con alguna proporción de compradores mayor a la actual

In [44]:
proporcion_compradores = 0.10

df_menos_no_compradores = resample(df_no_compradores, 
                                   replace=True,
                                   n_samples=int(df_no_compradores.label.value_counts()[0] * (1-proporcion_compradores)),
                                   random_state=123)
df_downsample = pd.concat([df_compradores, df_menos_no_compradores])

## Entrenando y comparando

La idea ahora es entrenar ambos sets que armamos y comparar con AUC-Score de cada uno

#### Primero vamos a entrenar con el set de datos con compras repetidas (Up-Sample)

In [46]:
features = df_upsample.columns.tolist()
features.remove('label')

X_train, X_test, Y_train, Y_test = train_test_split(df_upsample[features],
                                                    df_upsample['label'], 
                                                    test_size=0.20, 
                                                    random_state=123
                                                   )

In [47]:
rf_us = RandomForestClassifier(n_estimators=60, n_jobs=2, min_samples_split=200,
                             random_state=123, class_weight='balanced', max_features=0.6)
rf_us.fit(X_train,Y_train)

# vemos como le va en el AUC-Score
Y_pred = rf_us.predict_proba(X_test)
Y_pred_proba = [p[1] for p in Y_pred]
print(roc_auc_score(Y_test, Y_pred_proba))

0.9398446900858614


Ahí tenemos el valor del AUC-Score que obtuvimos con el set con compras duplicadas.

Ahora vamos a ver otras métricas que también nos importan.

In [48]:
print(classification_report(Y_test, rf_us.predict(X_test)))

             precision    recall  f1-score   support

          0       0.94      0.81      0.87      3646
          1       0.71      0.91      0.80      1885

avg / total       0.86      0.84      0.85      5531



#### Ahora vamos a entrenar con el set donde quitamos algunas no compras (Down-Sample)

In [49]:
features = df_downsample.columns.tolist()
features.remove('label')

X_train, X_test, Y_train, Y_test = train_test_split(df_downsample[features],
                                                    df_downsample['label'], 
                                                    test_size=0.20, 
                                                    random_state=123
                                                   )

In [50]:
rf_ds = RandomForestClassifier(n_estimators=30, n_jobs=2, min_samples_split=200,
                             random_state=123, class_weight='balanced')
rf_ds.fit(X_train,Y_train)

# vemos como le va en el AUC-Score
Y_pred = rf_ds.predict_proba(X_test)
Y_pred_proba = [p[1] for p in Y_pred]
print(roc_auc_score(Y_test, Y_pred_proba))

0.8508168494581538


Vemos como claramente empeoró el AUC-Score. Ahora veremos como le fué en las otras metricas que estabamos evaluando.

In [51]:
print(classification_report(Y_test, rf_ds.predict(X_test)))

             precision    recall  f1-score   support

          0       0.98      0.80      0.88      3330
          1       0.17      0.74      0.28       184

avg / total       0.94      0.80      0.85      3514



## Hacer un submit

Ahora que comparamos ambas formas de balancear los datos, quiero ver como le va a ir en el submit de kaggle.

Voy a usar el dataset con compras repetidas (Up-Sample) ya que es el que mejor AUC-Score me dió.

In [33]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_features, how='inner')
df_submit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19415 entries, 4886f805 to 80aea0a0
Empty DataFrame

In [34]:
kaggle_pred = rf_us.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [35]:
df_submit.to_csv('submit.csv')